In [ ]:
%sql
SELECT
    aufk.aufnr AS ordem,
    aufk.auart AS tipo_ordem,
    aufk.ktext AS texto_breve,
    aufk.erdat AS data_criacao,
    aufk.bukrs AS empresa,
    aufk.werks AS centro_trabalho,
    afih.equnr AS equipamento,
    iloa.tplnr AS local_instalacao
FROM poseidon_uc.saps4hanarawview.aufk AS aufk
LEFT JOIN poseidon_uc.saps4hanarawview.afih AS afih
    ON afih.aufnr = aufk.aufnr
LEFT JOIN poseidon_uc.saps4hanarawview.iloa AS iloa
    ON iloa.iloan = afih.iloan
WHERE 
    aufk.auart IN ('GIA','GAA','GRA')
    AND SUBSTRING(iloa.tplnr, 1, 4) IN ('GHIN','GHEN','GHSM')
ORDER BY aufk.erdat DESC, aufk.aufnr
LIMIT 1000


In [ ]:
%sql
WITH orders_ok AS (
    SELECT DISTINCT afih.aufnr
    FROM poseidon_uc.saps4hanarawview.afih afih
    JOIN poseidon_uc.saps4hanarawview.iloa iloa
      ON iloa.iloan = afih.iloan
    WHERE SUBSTRING(iloa.tplnr, 1, 4) IN ('GHIN','GHEN','GHSM')
),
afih_pick AS (
    SELECT
        afih.aufnr,
        MAX(afih.equnr) AS equnr,
        MAX(afih.iloan) AS iloan
    FROM poseidon_uc.saps4hanarawview.afih afih
    WHERE afih.aufnr IN (SELECT aufnr FROM orders_ok)
    GROUP BY afih.aufnr
)
SELECT
    k.aufnr AS ordem,
    k.auart AS tipo_ordem,
    k.ktext AS texto_breve,
    k.erdat AS data_criacao,
    k.bukrs AS empresa,
    k.werks AS centro_trabalho,
    p.equnr AS equipamento,
    i.tplnr AS local_instalacao
FROM poseidon_uc.saps4hanarawview.aufk k
JOIN orders_ok ok ON ok.aufnr = k.aufnr
LEFT JOIN afih_pick p ON p.aufnr = k.aufnr
LEFT JOIN poseidon_uc.saps4hanarawview.iloa i ON i.iloan = p.iloan
WHERE 
    k.auart IN ('GIA','GAA','GRA')
ORDER BY k.erdat DESC, k.aufnr
LIMIT 1000



In [ ]:
%sql
-- Extração de Ordens de Manutenção (similar à IW38)
-- Filtra por tipo de ordem (GIA, GAA, GRA) e local de instalação (GHIN, GHEN, GHSM)
-- Garante uma linha por ordem (aufnr único)

WITH ordens_validas AS (
    -- Identifica ordens que possuem locais de instalação válidos
    SELECT DISTINCT afih.aufnr
    FROM poseidon_uc.saps4hanarawview.afih AS afih
    INNER JOIN poseidon_uc.saps4hanarawview.iloa AS iloa
        ON iloa.iloan = afih.iloan
    WHERE SUBSTRING(iloa.tplnr, 1, 4) IN ('GHIN', 'GHEN', 'GHSM')
),
dados_afih AS (
    -- Seleciona dados principais do AFIH (1 registro por ordem)
    -- Em caso de múltiplos registros, pega o mais recente/relevante
    SELECT
        afih.aufnr,
        MAX(afih.equnr) AS equipamento,
        MAX(afih.iloan) AS iloan,
        MAX(afih.iwerk) AS planta_manutencao,
        MAX(afih.ingpr) AS grupo_planejador,
        MAX(afih.ilart) AS tipo_atividade,
        MAX(afih.qmnum) AS numero_notificacao
    FROM poseidon_uc.saps4hanarawview.afih AS afih
    WHERE afih.aufnr IN (SELECT aufnr FROM ordens_validas)
    GROUP BY afih.aufnr
)
SELECT
    aufk.aufnr AS ordem,
    aufk.auart AS tipo_ordem,
    aufk.ktext AS texto_breve,
    aufk.erdat AS data_criacao,
    aufk.ernam AS criado_por,
    aufk.aedat AS data_alteracao,
    aufk.aenam AS alterado_por,
    aufk.bukrs AS empresa,
    aufk.werks AS centro_trabalho,
    aufk.kostl AS centro_custo,
    aufk.waers AS moeda,
    aufk.loekz AS ind_exclusao,
    afih.equipamento,
    iloa.tplnr AS local_instalacao,
    iloa.stort AS desc_local_instalacao,
    afih.planta_manutencao,
    afih.grupo_planejador,
    afih.tipo_atividade,
    afih.numero_notificacao
FROM poseidon_uc.saps4hanarawview.aufk AS aufk
INNER JOIN ordens_validas AS ov
    ON ov.aufnr = aufk.aufnr
LEFT JOIN dados_afih AS afih
    ON afih.aufnr = aufk.aufnr
LEFT JOIN poseidon_uc.saps4hanarawview.iloa AS iloa
    ON iloa.iloan = afih.iloan
WHERE 
    aufk.auart IN ('GIA', 'GAA', 'GRA')  -- Filtra tipos de ordem
ORDER BY aufk.erdat DESC, aufk.aufnr
LIMIT 1000

In [ ]:
%sql
-- Extração COMPLETA da tabela AUFK (todos os campos)
-- Para análise detalhada dos registros duplicados
SELECT
    aufk.DI_SEQUENCE_NUMBER,
    aufk.DI_OPERATION_TYPE,
    aufk.MANDT AS cliente_sap,
    aufk.AUFNR AS ordem,
    aufk.AUART AS tipo_ordem,
    aufk.AUTYP AS tipo_automatico,
    aufk.REFNR AS numero_referencia,
    aufk.ERNAM AS usuario_criacao,
    aufk.ERDAT AS data_criacao,
    aufk.AENAM AS usuario_alteracao,
    aufk.AEDAT AS data_alteracao,
    aufk.KTEXT AS texto_breve,
    aufk.LTEXT AS texto_longo,
    aufk.BUKRS AS empresa,
    aufk.WERKS AS centro_trabalho,
    aufk.GSBER AS area_negocio,
    aufk.KOKRS AS area_controle,
    aufk.CCKEY AS chave_centro_custo,
    aufk.KOSTV AS centro_custo_verificacao,
    aufk.STORT AS local_armazenagem,
    aufk.SOWRK AS centro_trabalho_origem,
    aufk.ASTKZ AS indicador_estoque,
    aufk.WAERS AS moeda,
    aufk.ASTNR AS numero_estoque,
    aufk.STDAT AS data_inicio,
    aufk.ESTNR AS numero_estimativa,
    aufk.PHAS0 AS fase_0,
    aufk.PHAS1 AS fase_1,
    aufk.PHAS2 AS fase_2,
    aufk.PHAS3 AS fase_3,
    aufk.PDAT1 AS data_planejada_1,
    aufk.PDAT2 AS data_planejada_2,
    aufk.PDAT3 AS data_planejada_3,
    aufk.IDAT1 AS data_inicio_1,
    aufk.IDAT2 AS data_inicio_2,
    aufk.IDAT3 AS data_inicio_3,
    aufk.OBJID AS id_objeto,
    aufk.VOGRP AS grupo_venda,
    aufk.LOEKZ AS indicador_exclusao,
    aufk.PLGKZ AS indicador_planejamento,
    aufk.KVEWE AS area_uso,
    aufk.KAPPL AS aplicacao,
    aufk.KALSM AS esquema_calculo,
    aufk.ZSCHL AS chave_calculo,
    aufk.ABKRS AS area_abono,
    aufk.KSTAR AS elemento_custo,
    aufk.KOSTL AS centro_custo,
    aufk.SAKNR AS conta_contabil,
    aufk.SETNM AS nome_conjunto,
    aufk.CYCLE AS ciclo,
    aufk.SDATE AS data_inicio_ciclo,
    aufk.SEQNR AS numero_sequencia,
    aufk.USER0 AS campo_usuario_0,
    aufk.USER1 AS campo_usuario_1,
    aufk.USER2 AS campo_usuario_2,
    aufk.USER3 AS campo_usuario_3,
    aufk.USER4 AS campo_usuario_4,
    aufk.USER5 AS campo_usuario_5,
    aufk.USER6 AS campo_usuario_6,
    aufk.USER7 AS campo_usuario_7,
    aufk.USER8 AS campo_usuario_8,
    aufk.USER9 AS campo_usuario_9,
    aufk.OBJNR AS numero_objeto_status,
    aufk.PRCTR AS centro_lucro,
    aufk.PSPEL AS elemento_wbs,
    aufk.AWSLS AS esquema_avaliacao,
    aufk.ABGSL AS esquema_abono,
    aufk.EB_POST AS indicador_posting,
    aufk.TXJCD AS codigo_jurisdicao,
    aufk.FUNC_AREA AS area_funcional,
    aufk.SCOPE AS escopo,
    aufk.PLINT AS integracao_planejamento,
    aufk.KDAUF AS ordem_venda,
    aufk.KDPOS AS posicao_venda,
    aufk.AUFEX AS ordem_externa,
    aufk.IVPRO AS processo_inventario,
    aufk.LOGSYSTEM AS sistema_logico,
    aufk.FLG_MLTPS AS flag_multiplos_tipos,
    aufk.ABUKR AS empresa_abono,
    aufk.AKSTL AS centro_custo_atividade,
    aufk.SIZECL AS classe_tamanho,
    aufk.IZWEK AS proposito_investimento,
    aufk.UMWKZ AS indicador_manutencao,
    aufk.KSTEMPF AS template_custo,
    aufk.ZSCHM AS esquema_calculo_z,
    aufk.PKOSA AS elemento_custo_principal,
    aufk.ANFAUFNR AS ordem_anterior,
    aufk.PROCNR AS numero_processo,
    aufk.PROTY AS tipo_processo,
    aufk.RSORD AS ordem_reserva,
    aufk.BEMOT AS motivo_requisicao,
    aufk.ADRNRA AS endereco_requisicao,
    aufk.ERFZEIT AS hora_criacao,
    aufk.AEZEIT AS hora_alteracao,
    aufk.CSTG_VRNT AS variante_custeio,
    aufk.COSTESTNR AS numero_estimativa_custo,
    aufk.VERAA_USER AS usuario_verificacao,
    aufk.ZCOD_PROJ AS codigo_projeto,
    aufk.ZCOD_CONTR AS codigo_contrato,
    aufk.ZCOD_MOTIV AS codigo_motivo,
    aufk.ZCOD_TEMAT AS codigo_tematico,
    aufk.ZCOD_EFEITO AS codigo_efeito,
    aufk.ZCOD_VALOR AS codigo_valor,
    aufk.ZCOD_TP_CO AS tipo_codigo_co,
    aufk.ZCOD_TP_EN AS tipo_codigo_en,
    aufk.ZCOD_TP_AL AS tipo_codigo_al,
    aufk.ZCOD_OBS_TX AS observacao_texto,
    aufk.ZINTEG_SIT AS situacao_integracao,
    aufk.ZPROCES_SIT AS situacao_processo,
    aufk.ZZEMPREEND1 AS empreendimento_1,
    aufk.ZZATO_AUT1 AS ato_autorizacao_1,
    aufk.ZZFUNCAO_TRA1 AS funcao_trabalho_1,
    aufk.ZZCOD_MOD1 AS codigo_modelo_1,
    aufk.ZZCOD_RAP1 AS codigo_rap_1,
    aufk.ZZCOD_CONTRATO1 AS codigo_contrato_1,
    aufk.EEW_AUFK_PS_DUMMY AS dummy_eew_ps,
    aufk.VNAME AS nome_variante,
    aufk.RECID AS id_registro,
    aufk.ETYPE AS tipo_entidade,
    aufk.OTYPE AS tipo_objeto,
    aufk.JV_JIBCL AS classe_jv,
    aufk.JV_JIBSA AS area_jv,
    aufk.JV_OCO AS oco_jv,
    aufk.CPD_UPDAT AS atualizacao_cpd,
    aufk.AD01PROFNR AS numero_perfil_ad01,
    aufk.VAPLZ AS codigo_postal_venda,
    aufk.WAWRK AS centro_trabalho_venda,
    aufk.FERC_IND AS indicador_ferc,
    aufk.AUFK_STATUS AS status_ordem,
    aufk.CLAIM_CONTROL AS controle_reivindicacao,
    aufk.UPDATE_NEEDED AS atualizacao_necessaria,
    aufk.UPDATE_CONTROL AS controle_atualizacao,
    aufk.ODQ_CHANGEMODE AS modo_mudanca_odq,
    aufk.ODQ_ENTITYCNTR AS contador_entidade_odq,
    aufk.current_ts AS timestamp_atual,
    aufk.yearCurrentTs AS ano_timestamp,
    aufk.monthCurrentTs AS mes_timestamp,
    aufk.dayCurrentTs AS dia_timestamp,
    aufk.dataImportacao AS data_importacao
FROM poseidon_uc.saps4hanarawview.aufk AS aufk
WHERE
    YEAR(aufk.ERDAT) = 2025 AND
    aufk.AUART IN ('GIA','GAA','GRA') AND
    aufk.BUKRS IN ('GHIN', 'GHSM', 'EPEI')
ORDER BY aufk.AUFNR, aufk.DI_SEQUENCE_NUMBER


In [ ]:
%sql
SELECT
    aufk.AUFNR AS ordem,
    aufk.KTEXT AS descricao,
    aufk.ERDAT AS dt_data_ordem, 
    aufk.ERFZEIT hr_data_ordem,
    aufk.AUART AS tipo_ordem,
    aufk.AEDAT AS dt_data_ref_etl 
FROM poseidon_uc.sapep1rawview.aufk aufk
WHERE
    aufk.AUART IN ('GAA', 'GIA', 'GRA', 'GOA')
ORDER BY aufk.AUFNR

In [ ]:
%sql
-- Extração apenas do CABEÇALHO das ordens (sem duplicatas)
-- Usando apenas a tabela AUFK para evitar multiplicação de registros
SELECT
    aufk.aufnr AS ordem,
    aufk.auart AS tipo_ordem,
    aufk.ktext AS texto_breve,
    aufk.erdat AS data_criacao,
    aufk.aedat AS data_alteracao,
    aufk.ernam AS usuario_criacao,
    aufk.aenam AS usuario_alteracao,
    aufk.bukrs AS empresa,
    aufk.werks AS centro_trabalho,
    aufk.waers AS moeda,
    aufk.kostl AS centro_custo,
    aufk.kstar AS elemento_custo,
    aufk.objnr AS numero_objeto,
    aufk.pspel AS elemento_wbs,
    aufk.loekz AS indicador_exclusao
FROM poseidon_uc.saps4hanarawview.aufk AS aufk
WHERE
    YEAR(aufk.erdat) = 2025 AND
    aufk.auart IN ('GIA','GAA','GRA') AND
    aufk.bukrs IN ('GHIN', 'GHSM', 'EPEI')
ORDER BY ordem